# PSyclone tutorial: NEMO API Example 3

This example shows how

Let's continue with the code introduced in example 2 and create a schedule from it.

In [ ]:
code = '''program test
  implicit none
  integer, parameter :: jpi=10, jpj=10, jpk=10
  real, dimension(jpi,jpj,jpk) :: a,b
  integer :: ji,jj,jk
  call timer_start()
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        b(ji,jj,jk) = 0.0
      end do
    end do
  end do
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        a(ji,jj,jk) = b(ji,jj,jk)
      end do
    end do
  end do
  call timer_end()
  write (6,*) "HELLO"
end program test'''

In [ ]:
from fparser.common.readfortran import FortranStringReader
reader = FortranStringReader(code)
from fparser.two.parser import ParserFactory
parser = ParserFactory().create(std="f2003")
parse_tree = parser(reader)

from psyclone.psyGen import PSyFactory
psy = PSyFactory("nemo").create(parse_tree)

invoke = psy.invokes.invoke_list[0]
schedule = invoke.schedule

schedule.view()

Transformations: loop fuse? OpenMP, OpenACC

In [ ]:
from psyclone.transformations import OMPParallelLoopTrans
from psyclone.nemo import NemoKern
omp_trans = OMPParallelLoopTrans()
for loop in schedule.loops():
    kernels = loop.walk(NemoKern)
    if kernels and loop.loop_type == "lat":
        omp_trans.apply(loop)

In [ ]:
schedule.view()

In [ ]:
print psy.gen

Let's move on to the [next example](example4.ipynb)